In [13]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable

In [14]:
input_data = """
benchmark    |  genmc   |  genmc  |  hmc     |  hmc   |  cdschecker  |  cdschecker  |  wmc   |  wmc   |  xmm   |  xmm   |
reorder2/    |  1296    |  .10    |  1296    |  .10   |  503         |  0.03        |  1296  |  .06   |  1296  |  .18   |
singleton/   |  4       |  .04    |  24      |  .05   |  4           |  0.00        |  24    |  .01   |  4     |  .04   |
fib_bench/   |  465696  |  3.82   |  465696  |  5.68  |  *           |  *           |  *     |  *     |  *     |  *     |
szymanski/1  |  6       |  .04    |  6       |  .05   |  560         |  0.21        |  444   |  .07   |  +     |  +     |
szymanski/2  |  78      |  .13    |  78      |  .31   |  *           |  *           |  *     |  *     |  +     |  +     |
szymanski/3  |  1068    |  2.02   |  1068    |  6.22  |  *           |  *           |  *     |  *     |  +     |  +     |
peterson/10  |  39      |  .04    |  40      |  .04   |  306         |  0.05        |  1152  |  .19   |  53    |  .05   |
peterson/20  |  39      |  .04    |  40      |  .04   |  1006        |  0.10        |  3892  |  2.02  |  53    |  .04   |
peterson/30  |  39      |  .04    |  40      |  .04   |  2106        |  0.31        |  8232  |  8.06  |  53    |  .04   |
dekker-bnd/  |  55      |  .04    |  55      |  .04   |  244         |  0.15        |  55    |  .01   |  61    |  .05   |
sigma/       |  *       |  *      |  *       |  *     |  *           |  *           |  *     |  *     |  *     |  *     |
indexer/12   |  8       |  .04    |  8       |  .05   |  8           |  0.58        |  8     |  .02   |  8     |  .05   |
indexer/13   |  64      |  .08    |  64      |  .14   |  *           |  *           |  64    |  .05   |  64    |  .11   |
indexer/14   |  512     |  .44    |  512     |  .94   |  *           |  *           |  512   |  .30   |  512   |  .61   |
indexer/15   |  4096    |  3.38   |  4096    |  7.53  |  *           |  *           |  4096  |  2.44  |  4096  |  5.03  |
"""

In [15]:
def un_underscore(name: str) -> str:
    return name.replace("_", "\\_")

def parse_name(name: str) -> str:
    match un_underscore(name).split('/'):
        case [a]:
            return a
        case [a, ""]:
            return a
        case [a, number]:
            return f"{a}({number})"

def parse_result(r: str) -> str:
    match r:
        case '*':
            return '{\\fontspec{Symbola}\\symbol{"231B}}'
        case "+":
            return ''
        case r:
            return r

def parse_line(line: str) -> tuple[list[str], list[str]]:
    parts = [p.strip() for p in line.split('|') if p != ""]
    name = parse_name(parts[0])
    parts = [parse_result(p) for p in parts[1:]]
    execs = [name] + [parts[i] for i in range(0, len(parts), 2)]
    times = [name] + [parts[i] for i in range(1, len(parts), 2)]
    return execs, times

lines = [l for l in input_data.splitlines() if l != ""][1:]
tuples = [parse_line(line) for line in lines]
execs = [['Test Name', 'GenMC execs', ' HMC execs', 'CDSCheker execs', 'WMC execs', 'XMM execs']] + [exe for exe, time in tuples]
times = [['Test Name', 'GenMC time', 'HMC time', 'CDSChecker time', 'WMC time', 'XMM time']] + [time for exe, time in tuples]

In [16]:
# rows = [['Test Name', 'GenMC execs', ' HMC execs', 'CDSCheker execs', 'WMC execs', 'XMM execs']] + [[exe[0]] + [f"{e} ({t} s)" for e, t in zip(exe[1:], time[1:])] for exe, time in tuples]

# table_1 = Texttable()
# # table_1.set_cols_width(max_len)
# table_1.set_cols_align(["l"] + ["c" for i in range(len(execs[0]) - 1)])
# table_1.set_cols_valign(["m" for i in range(len(execs[0]))])
# table_1.add_rows(rows)
# # print(table_1.draw())
# # print('\nLatextable Output:')
# print(latextable.draw_latex(table_1, position='H', caption="Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-execs"))

In [19]:
# max_len = [max([len(row[i]) for row in rows]) for i in range(len(rows[0]))]

table_1 = Texttable()
# table_1.set_cols_width(max_len)
table_1.set_cols_align(["l"] + ["c" for i in range(len(execs[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(execs[0]))])
table_1.add_rows(execs)

table_2 = Texttable()
table_2.set_precision(2)
table_2.set_cols_align(["l"] + ["c" for i in range(len(times[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(times[0]))])
table_2.add_rows(times)

print(latextable.draw_latex(table_1, position='H', caption="Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-execs"))
print(latextable.draw_latex(table_2, position='H', caption="Execution time (seconds) on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-time"))

\begin{table}[H]
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|}
			\hline
			Test Name & GenMC execs &  HMC execs & CDSCheker execs & WMC execs & XMM execs \\
			\hline
			reorder2 & 1296 & 1296 & 503 & 1296 & 1296 \\
			\hline
			singleton & 4 & 24 & 4 & 24 & 4 \\
			\hline
			fib\_bench & 465696 & 465696 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} \\
			\hline
			szymanski(1) & 6 & 6 & 560 & 444 &  \\
			\hline
			szymanski(2) & 78 & 78 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			szymanski(3) & 1068 & 1068 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			peterson(10) & 39 & 40 & 306 & 1152 & 53 \\
			\hline
			peterson(20) & 39 & 40 & 1006 & 3892 & 53 \\
			\hline
			peterson(30) & 39 & 40 & 2106 & 8232 & 53 \\
			\hline
			dekker-bnd & 55 & 55 & 244 & 55 & 61 \\
			\hline
			sigma & {\fontspec{Symbola}\symbol{"231B}} & {\fo